In [1]:
import os
import tqdm
import torch
import torchvision
from deepface import DeepFace
from lib import Autoencoder, FacesDataset, NoiseScheduler,unet, Demorpher

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using device: {device}")

using device: cpu


In [3]:
encoder = torchvision.models.vgg16(pretrained=False) # autoencoder shape
encoder = encoder.features[:11]

autoencoder = Autoencoder(encoder)
autoencoder.load_state_dict(torch.load('ae_Casia_30.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [4]:
batch_size = 1

In [6]:
diffusion = unet(1)
diffusion.load_state_dict(torch.load('diffusion_65.pth',map_location=torch.device('cpu')))

<All keys matched successfully>

In [7]:
scheduler = NoiseScheduler(0.01,0.05,20) 

In [36]:
model = Demorpher(autoencoder.get_encoder(), autoencoder.get_decoder(), diffusion,scheduler)
model.eval()

Demorpher(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1): Sequential(
      (0): Conv2d(256, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01, inplace=True)
      (2): Conv2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (3): LeakyReLU(

In [9]:
FacesDataset = FacesDataset("Dataset/Neutral_Brightness", "Dataset/Morphed_Val_Brightness")

Found 1269 identities.


In [10]:
im_a,im_id,im_b, paths = FacesDataset[0]
im_a = im_a.unsqueeze(0).to(device)
im_id = im_id.unsqueeze(0).to(device)

In [11]:
im_a.shape

torch.Size([1, 3, 224, 224])

In [12]:
im_idout = model(im_a,im_id)

z_ID nan:  tensor(False)
 z_A nan:  tensor(False)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
noise nan:  tensor(True)
tensor([[[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]]]], grad_fn=<DivBackward0>)
torch.Size([1, 1, 56, 56])


In [ ]:
im_idout

In [ ]:
out = DeepFace.verify( img1_path = im_b.detach().numpy(), img2_path = im_idout.detach().numpy(), threshold=0.5)
out2 = DeepFace.verify( img1_path = im_a.detach().numpy(), img2_path = im_idout.detach().numpy(), threshold=0.5)

In [ ]:
out

In [ ]:
out2

In [ ]:
for im_a,im_id,im_b, paths in tqdm(FacesDataset):
    im_idout = model(im_a,im_id)
    out = DeepFace.verify( img1_path = im_b, img2_path = im_idout, threshold=0.5)
    